<a href="https://colab.research.google.com/github/CleverAndWitty/CIS1501-Fall2018/blob/master/LetterCount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 48.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=d5a687ca449887d3ed5927050061f12384b1a4e82811dbc0e5754b2f7adc8dfd
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import re #Regular Expressions

conf = SparkConf().setAppName('SparkWordCount')
sc = SparkContext.getOrCreate(conf = conf)

sqlContext = SparkSession.builder\
    .master("local")\
        .appName("Colab")\
            .config('spark.ui.port', '4050')\
                .getOrCreate()
        

Define Input File

In [12]:
input_file = sc.textFile('Sample.txt')

Function to return each letter in a string

In [32]:
alphabet = r"[a-zA-Z]"

def LettersInWord(StringInput):
  for i in range(len(StringInput)):
    if re.match(alphabet, StringInput[i]):
      return StringInput[i].lower()

Map by characters

In [33]:
Words = input_file.flatMap(
    lambda line: line.split())
Letters = Words.map(lambda word : LettersInWord(word), Words)
LetterCount = Letters.map(lambda letter : (letter , 1))
countKeys = LetterCount.reduceByKey(lambda a, b : a + b)

TypeError: ignored

Create the DataFrame and display it

In [58]:
from pyspark.sql.types import NullType
dfLetterCounts = sqlContext.createDataFrame(countKeys)\
.withColumnRenamed('_1', 'Letter')\
.withColumnRenamed('_2', 'Count')
dfLetterCounts = dfLetterCounts.filter(dfLetterCounts.Letter != 'null')
dfLetterCounts.sort(dfLetterCounts.Letter).show()

+------+------+
|Letter| Count|
+------+------+
|     a| 85667|
|     b| 45597|
|     c| 34694|
|     d| 29830|
|     e| 19116|
|     f| 36930|
|     g| 21021|
|     h| 60736|
|     i| 62305|
|     j|  3354|
|     k|  9491|
|     l| 29697|
|     m| 55800|
|     n| 26834|
|     o| 43590|
|     p| 27860|
|     q|  2378|
|     r| 14396|
|     s| 66069|
|     t|126395|
+------+------+
only showing top 20 rows



Output CSV

In [29]:
dfLetterCounts.toPandas().to_csv('LetterCounts.csv', index=False)


The Dead Code

In [ ]:

"""Attempt 2
Words = input_file.flatMap(
    lambda line: line.split())
Words = Words.map(lambda letter : letter, Words)
Letters = Words.map(lambda letter : letter[x], lambda x : x, Words[x])
LetterCount = Letters.map(lambda letter : (letter , 1))
countKeys = LetterCount.reduceByKey(lambda a, b : a + b)
"""

"""Attempt 1
Words = input_file.flatMap(
    lambda line: line.split())
    #re.findall(r"[a-zA-Z]",line.split()))
Letters = Words.map(lambda word :
                      re.findall(r"[a-zA-Z]", word))
Letters = LetterObj.map(lambda letter : 
                        letter.match)
LetterCount = Words.map(lambda letter: (letter, 1))
countKeys = LetterCount.reduceByKey(lambda a, b: a + b)
FiveWordPhrase = Words.map(lambda phrase : (phrase[0] + phrase[1] + phrase[2] + phrase[3] + phrase[4]) if len(phrase) >=5 else phrase)
"""